In [1]:
import pandas as pd
import os
import re
import codecs
from IPython.display import display
from six.moves import cPickle as pickle
import string
from PIL import Image
import numpy as np
import h5py

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [5]:
width = None
pd.options.display.max_rows = 600
pd.options.display.max_columns = width
pd.options.display.max_colwidth = 600
pd.options.display.width = width
pd.options.display.max_seq_items = None
pd.options.display.expand_frame_repr = False
pd.options.display.colheader_justify = 'left'

In [24]:
class Visualize(object):
    def __init__(self, storedir, gen_datadir):
        self._storedir = storedir
        self._word2id = pd.read_pickle(os.path.join(gen_datadir, 'dict_vocab.pkl'))
        i2w = pd.read_pickle(os.path.join(gen_datadir, 'dict_id2word.pkl'))
        for i in range(-1,-11,-1):
            i2w[i] = '%d'%i
        self._id2word = {}
        ## Append one space after each token except backslash
        for k in i2w.keys():
            self._id2word[k] = i2w[k] + " "
        self._id2word[self._word2id['id']['\\']] = '\\'
        
    
    @property
    def w2i(self):
        return self._word2id['id']

    @property
    def i2w(self):
        return self._id2word
    
    def np(self, graph, step, key):
        """
        Args:
            graph: 'training' or 'validation'
            step:  step who's output is to be fetched
            key:   key of object to fetch - e.g. 'predicted_ids'
        """
        h5 = h5py.File(os.path.join(self._storedir, '%s_%d.h5'%(graph,step)))
        return h5[key][...]
    
    def df(self, graph, step, key):
        return pd.DataFrame(self.np(graph, step, key))
    
    def words(self, graph, step, key):
        df = self.df(graph, step, key)
        return df.applymap(lambda x: self._id2word[x])

    def str(self, graph, step, key):
        df_str = self.words(graph, step, key)
        
        ## each token's string version - excepting backslash - has a space appended to it,
        ## therefore the string output should be compile if the prediction was syntactically correct
        return pd.DataFrame(["".join(row) for row in df_str.itertuples(index=False)])
                

In [25]:
v = Visualize('./tb_metrics_dev/2017-10-05 17-07-01 PDT/store', '../data/generated2')

In [50]:
v.str('validation', 1, 'y')

,0
0,"\phi \left ( x ^ 0 + \tau , { \bf x } \right ) = U \left ( \phi , \pi \right ) ^ { - 1 } \phi \left ( x ^ 0 , { \bf x } \right ) U \left ( \phi , \pi \right ) , \label { t h r e e f o u r t y s e v e n a } \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
1,"\psi _ L ^ { m } \to \frac { 1 } { \sqrt { z } } \exp \left ( - \frac { 1 - 2 n e _ L / e } { 2 a m _ F } z \right ) \left ( C _ 1 \sin ( \omega z ) + C _ 2 \cos ( \omega z ) \right ) ~ , \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
2,E \; \; = \; \; \sum _ { n = 0 } ^ { q } { { ( - 1 ) } ^ n \over ( n + 1 ) { ( n ! ) } ^ 2 } \; \eta ^ n \tau ^ { n + 1 } F \label { e q : x 2 } \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos
3,"\delta A = ( \rho ^ + \partial _ + + \rho ^ - \partial _ - ) A + \partial _ + \rho ^ + A , \hskip 1 0 p t \delta \bar A = ( \rho ^ + \partial _ + + \rho ^ - \partial _ - ) \bar A + \partial _ - \rho ^ - A \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
4,"g ^ * g _ { \alpha \beta } \: = \: \left ( h ^ g _ { \alpha } \right ) \, \left ( g _ { \alpha \beta } \right ) \, \left ( h ^ g _ { \beta } \right ) ^ { - 1 } \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
5,"\frac { d } { d s } \langle x ^ \mu \rangle \frac { d } { d s } \langle x _ \mu \rangle = \frac { \langle \pi ^ \mu \rangle \langle \pi _ \mu \rangle } { \langle m _ { o p } \rangle ^ 2 } = 1 , \label { r t p c } \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
6,"M ( r ) = \exp \left [ - \int _ 0 ^ r d r ' \p ( r ' ) \right ] \int _ 0 ^ r d r ' \q ( r ' ) \\exp \left [ \int _ 0 ^ { r ' } d r ' ' \p ( r ' ' ) \right ] \, \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
7,~ \label { c h i v e v } < \chi > = \frac { \lambda _ { v 1 } + \lambda _ { h 1 } ( z _ v / z _ h ) ^ { - m ^ 2 / 8 } } { \lambda _ { v 2 } ^ { ( 0 ) } + \lambda _ 2 ^ { * ( 0 ) } } . \eos \eos \eos \eos \eos \eos \eos
8,K ^ d _ { ( h ) } = - { 1 \over \Sigma _ d } \partial _ h \left ( \int _ 0 ^ { + \infty } { e ^ { - h s } \over ( \rho ^ 2 + s ^ 2 ) ^ { d + 1 \over 2 } } d s \right ) ~ ~ . \label { A } \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos
9,M _ { x x } = { 1 \over { t ^ { D / 2 } } } \left [ E _ 0 ( x ) + E _ 1 ( x ) t + E _ 2 ( x ) t ^ 2 + . . . \right ] \label { M x x _ e x p a n d } \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos


In [49]:
df = v.words('validation', 1, 'y')
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80
0,\phi,\left,(,x,^,0,+,\tau,",",{,\bf,x,},\right,),=,U,\left,(,\phi,",",\pi,\right,),^,{,-,1,},\phi,\left,(,x,^,0,",",{,\bf,x,},\right,),U,\left,(,\phi,",",\pi,\right,),",",\label,{,t,h,r,e,e,f,o,u,r,t,y,s,e,v,e,n,a,},\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos
1,\psi,_,L,^,{,m,},\to,\frac,{,1,},{,\sqrt,{,z,},},\exp,\left,(,-,\frac,{,1,-,2,n,e,_,L,/,e,},{,2,a,m,_,F,},z,\right,),\left,(,C,_,1,\sin,(,\omega,z,),+,C,_,2,\cos,(,\omega,z,),\right,),~,",",\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos
2,E,\,;,\,;,=,\,;,\,;,\sum,_,{,n,=,0,},^,{,q,},{,{,(,-,1,),},^,n,\over,(,n,+,1,),{,(,n,!,),},^,2,},\,;,\eta,^,n,\tau,^,{,n,+,1,},F,\label,{,e,q,:,x,2,},\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos
3,\delta,A,=,(,\rho,^,+,\partial,_,+,+,\rho,^,-,\partial,_,-,),A,+,\partial,_,+,\rho,^,+,A,",",\hskip,1,0,p,t,\delta,\bar,A,=,(,\rho,^,+,\partial,_,+,+,\rho,^,-,\partial,_,-,),\bar,A,+,\partial,_,-,\rho,^,-,A,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos
4,g,^,*,g,_,{,\alpha,\beta,},\,:,=,\,:,\left,(,h,^,g,_,{,\alpha,},\right,),\,",",\left,(,g,_,{,\alpha,\beta,},\right,),\,",",\left,(,h,^,g,_,{,\beta,},\right,),^,{,-,1,},\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos
5,\frac,{,d,},{,d,s,},\langle,x,^,\mu,\rangle,\frac,{,d,},{,d,s,},\langle,x,_,\mu,\rangle,=,\frac,{,\langle,\pi,^,\mu,\rangle,\langle,\pi,_,\mu,\rangle,},{,\langle,m,_,{,o,p,},\rangle,^,2,},=,1,",",\label,{,r,t,p,c,},\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos
6,M,(,r,),=,\exp,\left,[,-,\int,_,0,^,r,d,r,',\,p,(,r,',),\right,],\int,_,0,^,r,d,r,',\,q,(,r,',),\,\exp,\left,[,\int,_,0,^,{,r,',},d,r,',',\,p,(,r,',',),\right,],\,",",\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos
7,~,\label,{,c,h,i,v,e,v,},<,\chi,>,=,\frac,{,\lambda,_,{,v,1,},+,\lambda,_,{,h,1,},(,z,_,v,/,z,_,h,),^,{,-,m,^,2,/,8,},},{,\lambda,_,{,v,2,},^,{,(,0,),},+,\lambda,_,2,^,{,*,(,0,),},},.,\eos,\eos,\eos,\eos,\eos,\eos,\eos
8,K,^,d,_,{,(,h,),},=,-,{,1,\over,\Sigma,_,d,},\partial,_,h,\left,(,\int,_,0,^,{,+,\infty,},{,e,^,{,-,h,s,},\over,(,\rho,^,2,+,s,^,2,),^,{,d,+,1,\over,2,},},d,s,\right,),~,~,.,\label,{,A,},\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos
9,M,_,{,x,x,},=,{,1,\over,{,t,^,{,D,/,2,},},},\left,[,E,_,0,(,x,),+,E,_,1,(,x,),t,+,E,_,2,(,x,),t,^,2,+,.,.,.,\right,],\label,{,M,x,x,_,e,x,p,a,n,d,},\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos,\eos


In [28]:
v.str('training', 1, 'y')

,0
0,"h _ { \mu \nu } ( k ) = h ^ { + } ( k ) e _ { \mu \nu } ^ { + } ( k ) + h ^ { \times } ( k ) \, e _ { \mu \nu } ^ { \times } ( k ) \, \label { f a s } \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
1,"\label { 8 } D ^ { + + } _ r \Gamma ^ { ( 0 | 2 , 2 , 2 , 2 ) } = 0 \; , \quad \mbox { i f } x _ r \neq x _ s \; , \quad r , s = 1 , \ldots , 4 \; . \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
2,"\omega _ 0 ^ 2 - k _ n ^ 2 \left [ { 1 \over g } + { 1 \over 2 } k _ n ^ { D - 4 } \int ^ { \Lambda / k _ n } { d ^ D q \over ( 2 \pi ) ^ D } \, { 1 \over q ^ 2 ( q + 1 ) ^ 2 } \right ] = 0 . \eos \eos \eos \eos \eos"
3,"S _ { Y M } = \int d ^ d x \, \left ( - \frac { 1 } { 4 g ^ 2 } \right ) F _ { \mu \nu } * F ^ { \mu \nu } \, , \label { e q : N Y M _ a c t i o n } \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
4,"\label { A 1 3 } T ( u _ E ) = \left ( \begin { a r r a y } { c | c } 1 & b ( u ^ 0 _ E ) \vec u _ E ^ { \rm T } \\[ 1 e x ] \hline 0 & I \end { a r r a y } \right ) , \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
5,"K ^ { ( 1 ) } \; : = \; \frac { 1 } { - \triangle + \frac { e ^ 2 N } { \pi + g N } } \; = \; \frac { \pi + g N } { \pi } K _ { 1 1 } \; , \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
6,\label { Q r e p r e s e n t _ i n t } Q _ { \nu - 1 / 2 } ( \cosh \rho ) = \frac { 1 } { \sqrt { 2 } } \int _ { \rho } ^ { \infty } d t \frac { e ^ { - \nu t } } { \sqrt { \cosh t - \cosh \rho } } \. \eos \eos \eos \eos \eos \eos \eos
7,\left . - 2 \int d \phi d \theta N \\delta \left ( \gamma - \frac { \gamma ^ 3 } { l ^ 2 } \right ) \right | ^ { r _ { + + } } _ { r _ + } \. \label { v a r 3 } \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos
8,"S ^ { F } = \sum ^ { + \infty } _ { n _ { 1 } = 0 } \sum ^ { n _ { 1 } } _ { n _ { 0 } = - n _ { 1 } - 1 } \oplus ( 1 / 2 + n _ { 0 } , 3 / 2 + n _ { 1 } ) , \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
9,e q . ( 1 6 ) \bar { \bar { \Psi } } _ 0 \i \gamma ^ \mu \partial _ \mu \Psi _ 0 = \bar { \Psi } \i \gamma ^ \mu ( \partial _ \mu \Psi - \i e A _ \mu ) \Psi - m \bar { \Psi } \Psi \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos


In [21]:
v.w2i['\\']

61